In [10]:
#Operaciones para la BD.
from __future__ import division
import psycopg2
import sys, nltk, re, string, heapq, gensim
from nltk.corpus   import stopwords
from psycopg2 import Error
from nltk.tokenize import TweetTokenizer
from collections   import Counter
from prettytable   import PrettyTable
from textblob      import TextBlob
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from gensim import corpora, models
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import pandas.io.sql as sqlio
from optimus import Optimus
from translate import Translator

#nltk.download('stopwords')
sw = stopwords.words('spanish')
lemma = WordNetLemmatizer()
op = Optimus()

def crearConexion():
    
    connection = psycopg2.connect(user = "m.soto.montesinos",
                                      password = "1234",
                                      host = "127.0.0.1",
                                      port = "5432",
                                      database = "twitterBBVA")

    return connection

def obtenerTweets():

    try:
        
        connection = crearConexion()

        query = "select status_id, text from training_set"
        
        #DataFrame
        tweets = sqlio.read_sql_query(query, connection)
        
        #Array de tweets
        #cursor = connection.cursor()
        #cursor.execute(query)
        #tweets = cursor.fetchall()
        
    except (Exception, psycopg2.DatabaseError) as error :
    
        print ("Error obteniendo los datos", error)
    
    finally:
        
        #Cerramos la conexión de la base de datos.
        if (connection):
            
            #cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

    return tweets

def limpiarYTokenizarTweet(tweet):
    
    tweet   = re.sub(r'\$\w*', '', tweet)  # Remove tickers
    tweet   = re.sub(r'http?:.*$', '', tweet)
    tweet   = re.sub(r'https?:.*$', '', tweet)
    tweet   = re.sub(r'pic?.*\/\w*', '', tweet)
    tweet   = re.sub(r'[' + string.punctuation + ']+', ' ', tweet)  # Remove puncutations like 's
    
    tokens = TweetTokenizer(strip_handles = True, reduce_len = True).tokenize(tweet)
    tokens = [w.lower() for w in tokens if w not in sw and len(w) > 2 and w.isalpha()]
    tokens = [lemma.lemmatize(word) for word in tokens]
    
    return " ".join(tokens)

def analisisSentimiento(tweet):

    #translator = Translator(from_lang="es", to_lang="en")
    #translation = translator.translate(tweet)
    analysis  = TextBlob(tweet)
    sentiment = analysis.sentiment.polarity

    return sentiment
    

def main():

    tweets = obtenerTweets()
    sentimentList = []
    
    for index, row in tweets.iterrows():
        
        tweet =(row['text'])
        cleanTweet = limpiarYTokenizarTweet(tweet)
        sentiment = analisisSentimiento(cleanTweet)
        sentimentList.append(sentiment)
        
    tweets['sentimentOverall'] = sentimentList
    print(tweets)
    

if __name__ == '__main__':
    main()


PostgreSQL connection is closed
                status_id                                               text  \
0     1230639193950507008  @delaguila_01 @BBVA_Mex A mi me paso lo mismo ...   
1     1230619764365631488  @CESARRODORODRIG Ese es el punto Men tampoco l...   
2     1230489153747341312  @BBVARe_mx Lo comparto por aquí mismo. Lo únic...   
3     1230638903037591552  El BBVA hizo 52 pagos durante 14 años en la pr...   
4     1230638807147479046  Beto y Enrique cantándole a Ban... BBVA https:...   
5     1230638627828441088  FYI, LOL, BTW, ASAP, ¿qué significan estas pal...   
6     1230638073580605440  Los dichos y los hechos . https://t.co/gSgBKnkYhJ   
7     1230527432333168642  Dice la @aprensamadrid: "Exigimos a las compañ...   
8     1230637469328138241  I will never bank with BBVA again. I’ve been a...   
9     1230636926593531905                @DijoNicho Está más chido @BBVA_Mex   
10    1230619139380850693  @BBVARe_mx Lo sé, esa es la gran problemática ...   
11    12